In [1]:
import numpy as np
import torch
import torch.nn as nn
from scipy import interpolate
import matplotlib.pyplot as plt
from tqdm import tqdm

from vanilla_fem import *
from pod_fem import *
from deim_fem import *
from neim_fem import *
from other_functions import *

## Parameters
The parameters which we will let vary are $a,b,c$ so that
$$
    (a, b, c) \in \mathbb{P} = [-0.5, 0.5]\times[-0.5, 0.5]\times[1,2].
$$

In [2]:
# model parameters
parameters = []
for a in np.linspace(-0.5, 0.5, 2):
    for b in np.linspace(-0.5, 0.5, 2):
        for c in np.linspace(1, 2, 2):
            parameters.append((a, b, c))

A0 = 500
M = 1
L = 0.1

# number of time steps and final time
Nt = 200
t_final = 5
tVals = np.linspace(0, t_final, Nt)
dt = tVals[1] - tVals[0] # delta t

In [3]:
# define the mesh in space
mesh, interior_point_coords = get_triangulation(refinements=5)

# Get the finite element space, stiffness matrix, lumped mass matrix as a vector, and mass matrix
Vh, stiffness_matrix, gamma, mass_matrix = calculate_basis_integrals(mesh)

## Solve High Fidelity PDE
Note that we only keep track of nodes in the interior of $\Omega$ because we use homogeneous Dirichlet boundary conditions.

In [4]:
hf_solutions = []
for (a, b, c) in parameters:
    # Get vectors for the components of matrices (flattened over space) 
    # over time with time 0 initialized.
    Q1, Q2, p1, p2, r, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)

    # solve the PDE (updates the entries of Q1, Q2, etc. within the function)
    solve_Q_flow(Q1, Q2, p1, p2, r, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)
    
    hf_solutions.append((Q1, Q2, p1, p2, r))

Computing scheme...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:02<00:00, 90.77it/s]


Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 100.40it/s]


Computing scheme...


...

100%|███████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:03<00:00, 56.58it/s]


Computing scheme...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:03<00:00, 41.41it/s]


Computing scheme...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:02<00:00, 69.13it/s]


## Compute POD Matrices

In [5]:
Q1 = []
Q2 = []
p1 = []
p2 = []
r  = []
for sol in hf_solutions:
    Q1.append(sol[0])
    Q2.append(sol[1])
    p1.append(sol[2])
    p2.append(sol[3])
    r.append(sol[4])

Q1 = np.concatenate(Q1, axis=0)
Q2 = np.concatenate(Q2, axis=0)
p1 = np.concatenate(p1, axis=0)
p2 = np.concatenate(p2, axis=0)
r  = np.concatenate(r, axis=0)

max_rank = 5
U_Q1, U_Q2, U_r = get_POD_matrices(Q1, Q2, r, max_rank)

## Compare POD Solution to High Fidelity Solution (New Parameter)

In [6]:
a, b, c = -0.2, 0.2, 1.5

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod = initialize_Q_flow_POD(Q1_, Q2_, p1_, p2_, r_, U_Q1, U_Q2, U_r)
solve_Q_flow_POD(Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod, gamma, stiffness_matrix, U_Q1, U_Q2, U_r, Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_pod.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 107.36it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 1146.55it/s]


Relative Error: 0.02013503182385556


## Compute DEIM Operators

In [7]:
deim_modes = 7

nonlinearityQ = M * gamma.reshape(-1, 1) * np.concatenate([p1 * r, p2 * r], axis=0).T
UN_Q, nQ_indices = get_DEIM_operators(nonlinearityQ, deim_modes)
U_deimQ1 = U_Q1.T @ UN_Q
U_deimQ2 = U_Q2.T @ UN_Q

nonlinearityR = (2 * p1[:-1] * (Q1[1:] - Q1[:-1]) + 2 * p2[:-1] * (Q2[1:] - Q2[:-1])).T
UN_R, nR_indices = get_DEIM_operators(nonlinearityR, deim_modes)
U_deimR = U_r.T @ UN_R

## Compare DEIM Solution to High Fidelity Solution (New Parameter)

In [8]:
a, b, c = 0, 0, 1.5

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim = initialize_Q_flow_DEIM(Q1_, Q2_, p1_, p2_, r_, 
                                                                                          U_Q1, U_Q2, U_r, 
                                                                                          nQ_indices, nR_indices)
solve_Q_flow_DEIM(Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim, 
                    gamma, stiffness_matrix, 
                    U_Q1, U_Q2, U_r, U_deimQ1, U_deimQ2, U_deimR,
                    nQ_indices, nR_indices,
                    Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_deim.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:02<00:00, 91.78it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 2197.42it/s]


Relative Error: 0.022312408831505412


## Compute NEIM Operators

### $Q$ Equation NEIM

In [9]:
# get ro_sols
ro_sols = []
for sol in hf_solutions:
    Q1_, Q2_, p1_, p2_, r_ = sol
    ro_sols.append(np.concatenate([Q1_ @ U_Q1, Q2_ @ U_Q2, r_ @ U_r], axis=1))
ro_sols = np.array(ro_sols)
ro_sols = (ro_sols - np.mean(ro_sols, axis=(0,1), keepdims=True)) / np.std(ro_sols, axis=(0,1), keepdims=True)

# get f_NEIM
f_NEIM = []
for sol in hf_solutions:
    Q1_, Q2_, p1_, p2_, r_ = sol
    nonlinearityQ = M * np.concatenate([(gamma[None] * p1_ * r_) @ U_Q1, (gamma[None] * p2_ * r_) @ U_Q2], axis=1) 
    f_NEIM.append(nonlinearityQ)
f_NEIM = np.array(f_NEIM)

mu = np.array(parameters)

In [10]:
NQ_NEIM, _, _, _, _ = NEIM_Q(ro_sols, f_NEIM, mu, max_modes=3, train_loop_iterations=3000)

5 Max Error: 0.009154532448179646 Mean Error: 0.005583442398264272
0 0.030774256547853623
100 2.2259146772836362e-05
200 5.002017477812175e-06
300 2.202781191289701e-06
400 1.4141182422883311e-06
500 1.0297994887606164e-06
600 8.008745827338366e-07
700 6.515770539121243e-07
800 5.470352385718916e-07
900 4.69254351351953e-07
1000 4.0862830447504246e-07
1100 3.5978175778116637e-07
1200 3.1947960789483187e-07
1300 2.856124856164329e-07
1400 2.567195242936552e-07
1500 2.3175937793756822e-07
1600 2.0996900962646927e-07
1700 1.9078288557371426e-07
1800 1.7377334498141444e-07
1900 1.5860940579539017e-07
2000 1.4502871429604266e-07
2100 1.3282458558780056e-07
2200 1.2181801741278874e-07
2300 1.1186666450110218e-07
2400 1.0284743608316408e-07
2500 9.465181050999117e-08
2600 8.718993616303085e-08
2700 8.038027831263697e-08
2800 7.415544984364788e-08
2900 6.845446225846201e-08
Mean Already Selected Error: 6.321968303618378e-07
1 Max Error: 0.006654595298111922 Mean Error: 0.003446348384850626
0 0

In [11]:
parameters[5]

(0.5, -0.5, 2.0)

In [12]:
[(np.linalg.norm(NQ_NEIM(np.array([0.5, -0.5, 2]), ro_sols[5, time]) - f_NEIM[5, time]) / np.linalg.norm(f_NEIM[5, time]),
  np.linalg.norm(NQ_NEIM(np.array([0.5, -0.5, 2]), ro_sols[5, time]) - f_NEIM[5, time])) for time in range(ro_sols.shape[1])]

[(6.992923576208635e-05, 8.926301203694962e-05),
 (0.00033970801418081677, 0.00010883746283748838),
 (0.007480221836023758, 0.0014870406725025413),
 (0.023938283946893908, 0.0033733221510444185),
 (0.020553995408609416, 0.002196885390426954),
 (0.029992284035557877, 0.0025396895221943532),
 (0.018749580536976354, 0.0012972876173916472),
 (0.03571666965241496, 0.0020667674443150554),
 (0.03905837164197238, 0.00192503182131887),
 (0.029100012145227717, 0.0012397411286914125),
 (0.03207705396980202, 0.0011957837831280218),
 (0.04746028672861149, 0.001564164458494476),
 (0.05825884500410389, 0.00171252152634577),
 (0.05997595986296105, 0.0015844927354457048),
 (0.05423152668900792, 0.0012963052644523203),
 (0.047123963556386955, 0.0010251888323817133),
 (0.048568499805956816, 0.0009667293987005934),
 (0.062120597680021206, 0.0011366254077530206),
 (0.08085255936650945, 0.0013656747487986545),
 (0.09803799111243343, 0.0015345650292284895),
 (0.1098370173264008, 0.0015987888580554182),
 (0.1

### $r$ Equation NEIM

In [13]:
# get ro_sols
ro_sols = []
for sol in hf_solutions:
    Q1_, Q2_, p1_, p2_, r_ = sol
    ro_sols.append(np.concatenate([Q1_[1:] @ U_Q1, Q1_[:-1] @ U_Q1, Q2_[1:] @ U_Q2, Q2_[:-1] @ U_Q2], axis=1))
ro_sols = np.array(ro_sols)
ro_sols = (ro_sols - np.mean(ro_sols, axis=(0,1), keepdims=True)) / np.std(ro_sols, axis=(0,1), keepdims=True)

# get f_NEIM
f_NEIM = []
for sol in hf_solutions:
    Q1_, Q2_, p1_, p2_, r_ = sol
    nonlinearityR = (2 * p1_[:-1] * (Q1_[1:] - Q1_[:-1]) + 2 * p2_[:-1] * (Q2_[1:] - Q2_[:-1])) @ U_r
    f_NEIM.append(nonlinearityR)
f_NEIM = np.array(f_NEIM)

mu = np.array(parameters)

In [14]:
NR_NEIM, _, _, _, _ = NEIM_R(ro_sols, f_NEIM, mu, max_modes=3, train_loop_iterations=3000)

5 Max Error: 1.654826732187084 Mean Error: 0.8282263772287175
0 0.3339110140605574
100 0.07264636209891764
200 0.020355600443726835
300 0.004614347754191841
400 0.0007605511882182573
500 0.0001300589453830937
600 1.616229160772154e-05
700 6.0251403180420264e-06
800 6.165606255073617e-06
900 3.4349614688751008e-06
1000 2.949826976380148e-06
1100 2.6485666627051057e-06
1200 2.2977986292157463e-06
1300 2.048494265894392e-06
1400 1.8500540981666783e-06
1500 1.6820786393498626e-06
1600 1.5301232357156896e-06
1700 4.273869127723653e-06
1800 1.2940427577158602e-06
1900 1.2029895384594128e-06
2000 1.1162167042160013e-06
2100 1.0337612640067937e-06
2200 9.558181803395367e-07
2300 8.877486366475202e-07
2400 1.030973660866199e-06
2500 7.694122867576839e-07
2600 7.34290708996771e-07
2700 6.725112491926586e-07
2800 6.377273659746991e-07
2900 5.914025440516958e-07
Mean Already Selected Error: 2.9248750724644264e-06
1 Max Error: 0.48361345154060403 Mean Error: 0.20020640401434933
0 0.2765961675649285

In [17]:
parameters[4]

(0.5, -0.5, 1.0)

In [18]:
[(np.linalg.norm(NR_NEIM(np.array([0.5, -0.5, 1]), ro_sols[4, time]) - f_NEIM[4, time]) / np.linalg.norm(f_NEIM[4, time]),
  np.linalg.norm(NR_NEIM(np.array([0.5, -0.5, 1]), ro_sols[4, time]) - f_NEIM[4, time]),
  np.linalg.norm(f_NEIM[4, time]), np.linalg.norm(NR_NEIM(np.array([0.5, -0.5, 1]), ro_sols[4, time]))) for time in range(ro_sols.shape[1])]

[(4.496144710223541e-06, 2.79659391204726e-05, 6.219981989654905, 6.219977),
 (0.0010550455780968836, 0.0012026474112938209, 1.1399009069003303, 1.1397932),
 (0.011347159688928726, 0.005334299221919132, 0.47009995171952484, 0.47044477),
 (0.02311139599754188, 0.0057477538505134864, 0.24869782211013197, 0.24708004),
 (0.02484279071022772, 0.0037125144521111906, 0.1494403143114979, 0.15230533),
 (0.029046842643371554,
  0.0028302718539981305,
  0.09743819280970954,
  0.09670208),
 (0.03048993156807765, 0.0020516567589715278, 0.06728964787574569, 0.06643082),
 (0.025289007793815016,
  0.0012266247532170223,
  0.048504265695905255,
  0.048716817),
 (0.0469908636655118,
  0.0016986092543272466,
  0.036147649177470084,
  0.036296323),
 (0.03860161709923639, 0.0010680357391328289, 0.0276681605433042, 0.027257292),
 (0.04506447109424547,
  0.0009755306248863839,
  0.021647444232646387,
  0.021177735),
 (0.06383738059243861,
  0.0011012391833396032,
  0.017250695017866108,
  0.017229225),
 (0.0